In [1]:
import tensorflow as tf
import os
import sys
import go_datafile_reader
import model
import numpy as np
import model_eval

In [2]:
train_dir = "../data/input/train"
test_dir = "../data/input/test"
ckpt_path = "../data/working/test.ckpt"

In [3]:
train_files = []
for subdir, dirs, files in os.walk(train_dir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".dat"):
            train_files.append(filepath)

test_files = []
for subdir, dirs, files in os.walk(test_dir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".dat"):
            test_files.append(filepath)

print "num train: %d, num test: %d" %(len(train_files), len(test_files))

num train: 957, num test: 30


In [4]:
#note you may have to change the os limit for number of open files to use the RandomAccessFileReader
#you can do this with the command "sudo ulimit -n 20000"
#if sudo can't find the ulimit command try the following below
#sudo sh -c "ulimit -n 20000 && exec su $LOGNAME"

reader = go_datafile_reader.RandomAccessFileReader(train_files, board_size=9)
test_reader = go_datafile_reader.GoDatafileReader(test_files, board_size=9)

test_reader.num_epochs = 0
test_features = []
test_targets = []
test_move_numbers = []
while(test_reader.num_epochs == 0):
    test_move_numbers.append(test_reader.move_index)
    final_state, _, feature_cube = test_reader.read_sample()
    test_features.append(feature_cube)
    test_targets.append(final_state)

Initializing pointers in 957 datafiles, this may take a few minutes


In [5]:
print len(reader.open_files)

957


In [6]:
x, ownership = model.place_holders()
y_conv = model.model(x)
loss = model.loss_function(ownership, y_conv)
train_op = model.train_step(loss)

prediction = tf.round(y_conv)
correct_prediction = tf.equal(ownership, prediction)
correct_count = tf.reduce_sum(tf.cast(correct_prediction, "float"))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver(tf.all_variables())

ckpt_dir, _ = os.path.split(ckpt_path)
ckpt = tf.train.latest_checkpoint(ckpt_dir)
if ckpt is not None:
    print("restore from previous checkpoint")
    saver.restore(sess, ckpt)

InternalError: Failed to create session.

In [ ]:
training_accuracies = []
test_accuracies = []
for k in range(100000):
    x_batch, y_batch = reader.get_batch(50)
    _, loss_value, y_value = sess.run([train_op, loss, y_conv], feed_dict={x:x_batch, ownership:y_batch})
    if k % 10 == 0:
        acc = accuracy.eval(feed_dict={x:x_batch, ownership:y_batch})
        training_accuracies.append(acc)
        print k, loss_value, acc
            
    if k % 1000 == 0:
        test_accuracy = model_eval.test_accuracy(test_features, test_targets, x, ownership, correct_count)
        test_accuracies.append(test_accuracy)
        print "Test accuracy: %f" %test_accuracy
    if k % 1000 == 0:
        print "Saving Checkpoint..."
        saver.save(sess, ckpt_path)
    

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def rolling_mean(number_list, window = 20):
    means = np.zeros(len(number_list))
    for i in xrange(len(means)):
        sub_window = number_list[i-window+1:i+1]
        means[i] = np.mean(sub_window)
    return means

In [ ]:
print len(training_accuracies)
means = rolling_mean(test_accuracies, 20)
plt.plot(test_accuracies, label="Batch Accuracy")
plt.plot(means, label= "Rolling Mean")
plt.title("Good Data Mixing 5layer 64filters")
plt.xlabel("Batch Num / 10")
plt.ylabel("Accuracy")
plt.ylim([.6,1])
plt.legend(loc=3)
plt.show()

In [ ]:
accuracies = []
y_trues = []
y_preds = []
feature_cubes = []
for i in xrange(len(test_features)):
    if i%500 == 0:
        print i
    x_, y_ = test_features[i], test_targets[i]
    y_val, y_pred, acc = sess.run([y_conv, prediction, accuracy], feed_dict={x:[x_], ownership:[y_]})
    accuracies.append(acc)
    y_trues.append(np.reshape(y_, [19,19]))
    y_preds.append(np.reshape(y_pred[0], [19,19]))
    feature_cubes.append(x_)

In [ ]:
idx = 3500
y_pred = y_preds[idx]
y_true = y_trues[idx]
feature_cube = feature_cubes[idx]

print test_move_numbers[idx], accuracies[idx]
model_eval.print_info(feature_cube, y_pred, y_true)

    

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()
df["Move Num"] = test_move_numbers
df["Accuracy"] = accuracies
grouped = df.groupby("Move Num").aggregate(np.mean)


In [ ]:
grouped.plot()
plt.title("Accuracy by move number, 5layer 64filters")
plt.xlabel("Move Number")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
plot = plt.hist(df[df["Move Num"] == 250]["Accuracy"].values, bins = 40)
plt.show()